In [77]:
from pyspark.sql import *
import plotly.plotly as py
from pyspark.mllib.stat import Statistics
import numpy as np
py.sign_in('harshitshah', 'rdckn6jnj9')
from plotly.graph_objs import *
import pandas as pd
import requests
import matplotlib
from pyspark.ml.feature import Normalizer, StandardScaler
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

#read data into pandas dataframe
df = pd.read_csv('YearPredictionMSD.txt')
df.columns = ['year','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','c26','c27','c28','c29','c30','c31','c32','c33','c34','c35','c36','c37','c38','c39','c40','c41','c42','c43','c44','c45','c46','c47','c48','c49','c50','c51','c52','c53','c54','c55','c56','c57','c58','c59','c60','c61','c62','c63','c64','c65','c66','c67','c68','c69','c70','c71','c72','c73','c74','c75','c76','c77','c78','c79','c80','c81','c82','c83','c84','c85','c86','c87','c88','c89','c90']

#Statistics
df_sample = df.sample(100)
print df_sample.describe()

#histogram and box-plot for year
x0 = df['year']
trace = Data([Histogram(x=x0, histnorm='count', name='Year')])
trace0 = Box(x=x0, , name='Year_Boxplot')
data = Data([trace0])
data1 = Data([trace])
plot_url2 = py.plot(data1, filename='basic-histogram')
plot_url = py.plot(data, filename='basic-box-plot')

#backup year column as a new dataframe
df_year = df['year']

#drop year column
df.drop(df.columns[[0]], axis=1, inplace=True)

#normalize (z-score)
df_norm = (df - df.mean())/df.std()

#convert year values to binary    
df_year[df_year < 1965] = 0
df_year[df_year >= 1965] = 1

#append to main dataframe
df_norm['year'] = df_year

#split the data into train and test
df_train = df_norm[:463715]
df_test = df_norm[463716:515345]

#write dataframe to csv
df_train.to_csv('df_train_Q1.csv',header = False)
df_test.to_csv('df_test_Q1.csv',header = False)

#defining a function to split values
def parsePoint(line):
    values = [float(x) for x in line.split(',')]
    return LabeledPoint(values[91], values[1:90])

#reading csv file into rdd
rdd_train = sc.textFile("df_train_Q1.csv")
rdd_test = sc.textFile("df_test_Q1.csv")

#parsing data
parsedData_train = rdd_train.map(parsePoint)
parsedData_test = rdd_test.map(parsePoint)

#run SVMWithSGD
model = SVMWithSGD.train(parsedData_train, iterations=100)

labelsAndPreds = parsedData_train.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData_train.count())
print("Training Error - SVMWithSGD = " + str(trainErr))

labelsAndPreds1 = parsedData_test.map(lambda p: (p.label, model.predict(p.features)))
testErr = labelsAndPreds1.filter(lambda (v, p): v != p).count() / float(parsedData_test.count())
print("Testing Error - SVMWithSGD = " + str(testErr))


#run LogisticRegressionWithLBFGS
model_Log = LogisticRegressionWithLBFGS.train(parsedData_train, iterations=100)

# Evaluating the model on training data
labelsAndPreds2 = parsedData_train.map(lambda p: (p.label, model_Log.predict(p.features)))
trainErr1 = labelsAndPreds2.filter(lambda (v, p): v != p).count() / float(parsedData_train.count())
print("Training Error - Logistic Regression = " + str(trainErr1))

labelsAndPreds3 = parsedData_test.map(lambda p: (p.label, model_Log.predict(p.features)))
testErr1 = labelsAndPreds3.filter(lambda (v, p): v != p).count() / float(parsedData_test.count())
print("Testing Error - Logistic Regression = " + str(testErr1))


Training Error - SVMWithSGD = 0.381551168282
Testing Error - SVMWithSGD = 0.382834895793
Training Error - Logistic Regression = 0.40553141477
Testing Error - Logistic Regression = 0.408015030604


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

